In [68]:
import pandas as pd
import matplotlib.pyplot as plt

from matplotlib import style
plt.style.use('ggplot')

from sklearn.impute import SimpleImputer

In [69]:
def imputar_f (df,list_cat,SimpleImputer,pd):  
    columns = df.columns.values
    df_c=df[list_cat]
    df_n=df.loc[:,~df.columns.isin(list_cat)]

    imputer_n=SimpleImputer(strategy='median')
    imputer_c=SimpleImputer(strategy='most_frequent')

    imputer_n.fit(df_n)
    imputer_c.fit(df_c)

    X_n=imputer_n.transform(df_n)
    X_c=imputer_c.transform(df_c)

    df_n=pd.DataFrame(X_n,columns=df_n.columns)
    df_c=pd.DataFrame(X_c,columns=df_c.columns)

    df = pd.concat([df_n,df_c],axis=1)[columns]
    return df

# ***1. Data read***

In [70]:
DataNames = {'employee_survey_data':'employee_survey', 'general_data':'general', 'manager_survey_data':'manager_survey', 'retirement_info':'retirement'}

In [71]:
for key, value in zip(DataNames.keys(), DataNames.values()):
  globals()[f'df_{value}'] = pd.read_csv(f'https://raw.githubusercontent.com/santiagogz11/HRProject/main/data/{key}.csv', engine='python', sep = None)

In [72]:
df_general.sort_values(by=['EmployeeID'],ascending=1).head(2)

,Age,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeID,Gender,JobLevel,...,NumCompaniesWorked,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
0,51,Travel_Rarely,Sales,6,2,Life Sciences,1,1,Female,1,...,1.0,Y,11,8,0,1.0,6,1,0,0
1,31,Travel_Frequently,Research & Development,10,1,Life Sciences,1,2,Female,1,...,0.0,Y,23,8,1,6.0,3,5,1,4


In [73]:
df_employee_survey.sort_values(by=['EmployeeID'],ascending=1).head(2)

,EmployeeID,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance
0,1,3.0,4.0,2.0
1,2,3.0,2.0,4.0


In [74]:
df_manager_survey.sort_values(by=['EmployeeID'],ascending=1).head(2)

,EmployeeID,JobInvolvement,PerformanceRating
0,1,3,3
1,2,2,4


In [75]:
df_manager_survey.sort_values(by=['EmployeeID'],ascending=1).head(2)

,EmployeeID,JobInvolvement,PerformanceRating
0,1,3,3
1,2,2,4


# ***2. Null treatment***

In [76]:
for df in DataNames.values():
  print(f'\n ----- {df} ---- \n')
  print(globals()[f'df_{df}'].isnull().sum())


 ----- employee_survey ---- 

EmployeeID                  0
EnvironmentSatisfaction    25
JobSatisfaction            20
WorkLifeBalance            38
dtype: int64

 ----- general ---- 

Age                         0
BusinessTravel              0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
EmployeeCount               0
EmployeeID                  0
Gender                      0
JobLevel                    0
JobRole                     0
MaritalStatus               0
MonthlyIncome               0
NumCompaniesWorked         19
Over18                      0
PercentSalaryHike           0
StandardHours               0
StockOptionLevel            0
TotalWorkingYears           9
TrainingTimesLastYear       0
YearsAtCompany              0
YearsSinceLastPromotion     0
YearsWithCurrManager        0
dtype: int64

 ----- manager_survey ---- 

EmployeeID           0
JobInvolvement       0
PerformanceRating    0
dtype: int6

In [77]:
for df in DataNames.values():
  print(f'\n ----- {df} ----- \n')
  for column in globals()[f'df_{df}'].columns:
    print(f'\n \t----- {column} ----- \n')
    print(f"\t {globals()[f'df_{df}'][column].unique()}")


 ----- employee_survey ----- 


 	----- EmployeeID ----- 

	 [   1    2    3 ... 4408 4409 4410]

 	----- EnvironmentSatisfaction ----- 

	 [ 3.  2.  4.  1. nan]

 	----- JobSatisfaction ----- 

	 [ 4.  2.  1.  3. nan]

 	----- WorkLifeBalance ----- 

	 [ 2.  4.  1.  3. nan]

 ----- general ----- 


 	----- Age ----- 

	 [51 31 32 38 46 28 29 25 45 36 55 47 37 21 35 26 50 53 42 44 49 18 41 39
 58 33 43 52 27 30 54 40 23 48 57 34 24 22 56 60 19 20 59]

 	----- BusinessTravel ----- 

	 ['Travel_Rarely' 'Travel_Frequently' 'Non-Travel']

 	----- Department ----- 

	 ['Sales' 'Research & Development' 'Human Resources']

 	----- DistanceFromHome ----- 

	 [ 6 10 17  2  8 11 18  1  7 28 14  3  4 16  9  5 20 29 15 13 24 19 22 25
 21 26 27 12 23]

 	----- Education ----- 

	 [2 1 4 5 3]

 	----- EducationField ----- 

	 ['Life Sciences' 'Other' 'Medical' 'Marketing' 'Technical Degree'
 'Human Resources']

 	----- EmployeeCount ----- 

	 [1]

 	----- EmployeeID ----- 

	 [   1    2    3 ... 44

In [78]:
# Tratamiento de datos nulos en la base df_employee_survey
df_employee_survey = imputar_f(df_employee_survey, ['EnvironmentSatisfaction', 'JobSatisfaction', 'WorkLifeBalance'], SimpleImputer, pd)

In [79]:
# Tratamiento de datos nulos en la base df_general
df_general.NumCompaniesWorked.fillna(df_general[df_general.NumCompaniesWorked.notnull()].NumCompaniesWorked.median(), inplace = True)
df_general.TotalWorkingYears.fillna(df_general[df_general.TotalWorkingYears.notnull()].TotalWorkingYears.median(), inplace = True)

In [80]:
# Tratamiento de datos nulos en la base df_retirement
df_retirement[df_retirement.resignationReason.isnull()].retirementType.unique()

array(['Fired'], dtype=object)

In [81]:
# Tratamiento de datos nulos en la base df_retirement
df_retirement.resignationReason.fillna('Fired', inplace=True)

In [92]:
# Eliminar variables de valor unico
for df in DataNames.values():
  columns = []
  for column in globals()[f'df_{df}'].columns:
    if globals()[f'df_{df}'][column].unique().size == 1:
      columns.append(column)
  globals()[f'df_{df}'].drop(columns=columns, inplace=True)

#***3. Variable types***


In [96]:
for df in DataNames.values():
  print(f'\n ----- {df} ---- \n')
  print(globals()[f'df_{df}'].dtypes)


 ----- employee_survey ---- 

EmployeeID                 float64
EnvironmentSatisfaction    float64
JobSatisfaction            float64
WorkLifeBalance            float64
dtype: object

 ----- general ---- 

Age                          int64
BusinessTravel              object
Department                  object
DistanceFromHome             int64
Education                    int64
EducationField              object
EmployeeID                   int64
Gender                      object
JobLevel                     int64
JobRole                     object
MaritalStatus               object
MonthlyIncome                int64
NumCompaniesWorked         float64
PercentSalaryHike            int64
StockOptionLevel             int64
TotalWorkingYears          float64
TrainingTimesLastYear        int64
YearsAtCompany               int64
YearsSinceLastPromotion      int64
YearsWithCurrManager         int64
dtype: object

 ----- manager_survey ---- 

EmployeeID           int64
JobInvolvement       

In [104]:
employeeSurveyColumns = {'EmployeeID':'string','EnvironmentSatisfaction':'string', 'JobSatisfaction':'string', 'WorkLifeBalance':'string'}
df_employee_survey = df_employee_survey.astype(employeeSurveyColumns)

generalColumns = {'EmployeeID':'string'}
df_general = df_general.astype(generalColumns)

managerSurveyColumns = {'EmployeeID':'string'}
df_manager_survey = df_manager_survey.astype(managerSurveyColumns)

retirementColumns = {'EmployeeID':'string', 'retirementType':'string', 'resignationReason':'string'}
df_retirement = df_retirement.astype(retirementColumns)